In [14]:
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.embeddings import HuggingFaceEmbeddings
from langchain_community.vectorstores import FAISS
from groq import Groq
from apikey import api
import json

In [15]:
with open("dataset.json", "r", encoding="utf-8") as f:
    dataset = json.load(f)

data = ""
for item in dataset:
    data += f"Problem: {item['problem']}\n"
    data += f"Solution: {item['solution']}\n\n"

In [16]:
splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=200)
chunks = splitter.create_documents([data])

In [17]:
embeddings = HuggingFaceEmbeddings(model_name="BAAI/bge-small-en-v1.5")
vector_store = FAISS.from_documents(chunks, embeddings)

In [18]:
client = Groq(api_key=api)

In [19]:
def get_answer(question: str) -> str:
    retriever = vector_store.as_retriever(search_type="similarity", search_kwargs={"k": 8})
    retrieved_docs = retriever.invoke(question)
    context = "\n\n".join(doc.page_content for doc in retrieved_docs)

    prompt = f"""Câu hỏi: {question}
            Ngữ cảnh: {context}
            Yêu cầu: 
            - Hãy phân tích vấn đề và đưa ra lời giải bằng TIẾNG VIỆT
            - Giải thích từng bước giải quyết vấn đề
            - Nếu có code mẫu, hãy chú thích rõ ràng bằng tiếng Việt
            - Đưa ra các lưu ý về độ phức tạp và cách tối ưu (nếu có)
            Trả lời:"""

    response = client.chat.completions.create(
        model="llama3-70b-8192",
        messages=[
            {"role": "system", "content": """Bạn là trợ lý lập trình viên chuyên nghiệp, giao tiếp hoàn toàn bằng tiếng Việt.

            NGUYÊN TẮC TRẢ LỜI:
            - Luôn trả lời bằng tiếng Việt, rõ ràng và dễ hiểu
            - Phân tích kỹ yêu cầu bài toán trước khi đưa ra lời giải
            - Giải thích code chi tiết, có comment tiếng Việt cho từng phần quan trọng
            - Nêu rõ độ phức tạp và cách tối ưu thuật toán nếu có thể

            ĐỊNH DẠNG TRẢ LỜI:
            1. Phân tích yêu cầu bài toán
            2. Ý tưởng giải quyết 
            3. Code mẫu + giải thích
            4. Độ phức tạp và tối ưu"""},
            
            {"role": "user", "content": prompt}
        ],
        temperature=0.3,
        max_tokens=8192,
        top_p=0.8,
        stream=False
    )

    return response.choices[0].message.content

In [20]:
print("Chatbot: Xin chào! Bạn muốn hỏi gì?")

user_input = """
Lập trình bằng C++
Cho mảng số nguyên A\[] gồm N phần tử. Hãy tìm số xuất hiện nhiều nhất trong mảng.
Nếu có nhiều số có cùng số lần xuất hiện nhiều nhất thì chọn số nhỏ nhất trong số đó.

Đầu vào:
* Dòng đầu tiên là số nguyên dương N (số lượng phần tử trong mảng).
* Dòng thứ hai gồm N số nguyên, là các phần tử của mảng A, các số cách nhau bởi một dấu cách.

Giới hạn:
1 ≤ N ≤ 10^5
-10^9 ≤ A\[i] ≤ 10^9

Đầu ra:
* In ra 2 số nguyên cách nhau một dấu cách:
* Số có số lần xuất hiện nhiều nhất trong mảng.
* Số lần xuất hiện của số đó.

Ví dụ:
Input:
5
1 2 2 1 3

Output:
1 2
"""

answer = get_answer(user_input)
print("Chatbot:", answer)

Chatbot: Xin chào! Bạn muốn hỏi gì?
Chatbot: **Phân tích vấn đề**

Cho mảng số nguyên A[] gồm N phần tử, hãy tìm số xuất hiện nhiều nhất trong mảng. Nếu có nhiều số có cùng số lần xuất hiện nhiều nhất thì chọn số nhỏ nhất trong số đó.

**Ý tưởng giải quyết**

Để giải quyết vấn đề này, chúng ta có thể sử dụng một cấu trúc dữ liệu là mảng đếm (counting array) để đếm số lần xuất hiện của mỗi số trong mảng. Sau đó, chúng ta sẽ tìm số có số lần xuất hiện nhiều nhất và chọn số nhỏ nhất trong số đó.

**Code mẫu**
```cpp
#include <iostream>
#include <vector>
using namespace std;

int main() {
    int N;
    cin >> N;
    vector<int> A(N);
    for (int i = 0; i < N; i++) {
        cin >> A[i];
    }

    // Tạo mảng đếm
    vector<int> count(1000001, 0); // Mảng đếm có kích thước 1000001 để chứa các số từ 0 đến 10^6
    for (int i = 0; i < N; i++) {
        count[A[i]]++;
    }

    // Tìm số có số lần xuất hiện nhiều nhất
    int maxCount = 0;
    int minNum = INT_MAX;
    for (int i = 0; i < 